In [1]:
from deepchem.molnet import load_qm7, load_delaney, load_qm8, load_qm9, load_lipo, load_freesolv
import deepchem as dc
from data_utils import prepare_datasets

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (C:\Users\user\anaconda3\envs\uq\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (C:\Users\user\anaconda3\envs\uq\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [2]:
def load_dataset():
    FEATURIZER = "coulomb"  # not ECFP -> will flatten (N, A, A) -> (N, A*A)
    # tasks, datasets, transformers = load_qm7()
    tasks, datasets, transformers = load_qm8()
    #
    # FEATURIZER = "ecfp"  # not ECFP -> will flatten (N, A, A) -> (N, A*A)
    # tasks, datasets, transformers = load_delaney()
    # tasks, datasets, transformers = load_lipo()

    train_dc, valid_dc, test_dc = prepare_datasets(datasets, featurizer_name=FEATURIZER)

    if train_dc.y.ndim == 2 and train_dc.y.shape[1] > 1:
        first_task_name = tasks[0]
        tasks = [first_task_name]

        def _keep_first_task(ds) -> dc.data.NumpyDataset:
            # y: (N, T) -> (N, 1) taking the first column
            return dc.data.NumpyDataset(
                X=ds.X,
                y=ds.y[:, 0:1],
                w=ds.w[:, 0:1],
                ids=ds.ids,
                n_tasks=1
            )

        train_dc = _keep_first_task(train_dc)
        valid_dc = _keep_first_task(valid_dc)
        test_dc = _keep_first_task(test_dc)

    return tasks, train_dc, valid_dc, test_dc, transformers


In [3]:
load_dataset()

(['E1-CC2'],
 <NumpyDataset X.shape: (17397, 676), y.shape: (17397, 1), w.shape: (17397, 1), task_names: [0]>,
 <NumpyDataset X.shape: (2175, 676), y.shape: (2175, 1), w.shape: (2175, 1), task_names: [0]>,
 <NumpyDataset X.shape: (2175, 676), y.shape: (2175, 1), w.shape: (2175, 1), task_names: [0]>,
 [<deepchem.trans.transformers.NormalizationTransformer at 0x18482483160>])

In [4]:
from main import main_nngp_exact, main_nngp_exact_ensemble_all, main_nngp_svgp, main_nngp_svgp_exact_ensemble_all

In [8]:
main_nngp_exact()

Train X shape: (3360, 1024)
Train y shape: (3360, 1)
[Iter 001] Loss: 1.4296 | lengthscale: N/A | outputscale: 6.907e-01 | noise: 6.908e-01 | feat| μ_abs(avg)=1.49e-06, σ(avg/min/max)=9.85e-01/9.78e-01/9.91e-01 | μ[:5]=-1.55e-06 -7.95e-08 -1.33e-06 1.05e-06 -5.16e-07 | σ[:5]=9.81e-01 9.84e-01 9.82e-01 9.85e-01 9.84e-01 | BNμ(mean)=5.96e-03, BNσ(mean)=8.10e-01 | pred| μ(mean/min/max)=5.001e-03/5.001e-03/5.001e-03 | σ(mean/min/max)=8.312e-01/8.312e-01/8.312e-01
[Iter 040] Loss: 0.7628 | lengthscale: N/A | outputscale: 6.254e-01 | noise: 5.885e-01 | feat| μ_abs(avg)=1.99e-03, σ(avg/min/max)=8.37e-01/7.92e-01/9.09e-01 | μ[:5]=2.78e-03 4.77e-03 -9.21e-04 -9.34e-04 8.85e-04 | σ[:5]=8.30e-01 8.61e-01 8.28e-01 8.43e-01 8.46e-01 | BNμ(mean)=1.34e-01, BNσ(mean)=1.96e+00 | pred| μ(mean/min/max)=1.269e-01/1.269e-01/1.269e-01 | σ(mean/min/max)=7.909e-01/7.909e-01/7.909e-01
[Iter 080] Loss: 0.6499 | lengthscale: N/A | outputscale: 6.111e-01 | noise: 4.909e-01 | feat| μ_abs(avg)=4.54e-03, σ(avg/min/m

In [5]:
main_nngp_exact_ensemble_all()

Train X shape: (17397, 676)
Train y shape: (17397, 1)
[Ensemble] Training model 1/5
[Iter 001] Loss: 1.4417 | lengthscale: N/A | outputscale: 6.907e-01 | noise: 6.908e-01 | feat| μ_abs(avg)=1.21e-05, σ(avg/min/max)=9.95e-01/9.95e-01/9.95e-01 | μ[:5]=-2.69e-06 2.09e-05 -7.81e-06 8.75e-06 -1.44e-05 | σ[:5]=9.95e-01 9.95e-01 9.95e-01 9.95e-01 9.95e-01 | BNμ(mean)=2.08e-02, BNσ(mean)=8.20e-01 | pred| μ(mean/min/max)=-1.567e-02/-1.567e-02/-1.567e-02 | σ(mean/min/max)=8.390e-01/8.390e-01/8.390e-01
[Iter 040] Loss: 1.2622 | lengthscale: N/A | outputscale: 6.445e-01 | noise: 6.026e-01 | feat| μ_abs(avg)=2.16e-03, σ(avg/min/max)=8.79e-01/8.36e-01/9.45e-01 | μ[:5]=5.35e-03 -1.77e-05 -9.12e-05 -3.16e-04 -3.96e-03 | σ[:5]=8.81e-01 8.38e-01 8.71e-01 8.90e-01 9.45e-01 | BNμ(mean)=2.08e+00, BNσ(mean)=4.85e+00 | pred| μ(mean/min/max)=-9.961e-02/-9.961e-02/-9.961e-02 | σ(mean/min/max)=8.105e-01/8.105e-01/8.105e-01
[Iter 080] Loss: 1.1891 | lengthscale: N/A | outputscale: 6.764e-01 | noise: 5.010e-01 | 

[{'name': 'uniform',
  'weights': array([0.2, 0.2, 0.2, 0.2, 0.2]),
  'valid_mse': 0.8433794353093625,
  'test_mse': 0.8795083233725106,
  'uq_metrics': {'alpha': 0.05,
   'empirical_coverage': 0.9632183908045977,
   'avg_pred_std': 1.0519688456409209,
   'nll': 1.3631024928778241,
   'ce': 0.001666404795182565,
   'spearman_err_unc': 0.14728735531106088}},
 {'name': 'precision',
  'weights': array([0.19430937, 0.19106756, 0.20859398, 0.20026775, 0.20576133]),
  'valid_mse': 0.8427104187784993,
  'test_mse': 0.8782375400387133,
  'uq_metrics': {'alpha': 0.05,
   'empirical_coverage': 0.9636781609195403,
   'avg_pred_std': 1.0515757239406924,
   'nll': 1.362499300677667,
   'ce': 0.0016838896885495536,
   'spearman_err_unc': 0.14647655493374537}},
 {'name': 'mse',
  'weights': array([0.15886768, 0.16885412, 0.20747875, 0.23471004, 0.23008941]),
  'valid_mse': 0.8409386634415369,
  'test_mse': 0.8768128979818148,
  'uq_metrics': {'alpha': 0.05,
   'empirical_coverage': 0.9664367816091954

In [6]:
main_nngp_svgp()

Train X shape: (17397, 676)
Train y shape: (17397, 1)
[Iter 001] Loss: 1.9570 | lengthscale: N/A | outputscale: 6.907e-01 | noise: 6.958e-01 | feat| μ_abs(avg)=8.08e-08, σ(avg/min/max)=1.00e+00/9.99e-01/1.00e+00 | μ[:5]=6.32e-08 -1.02e-08 4.33e-07 2.14e-08 6.34e-07 | σ[:5]=9.99e-01 1.00e+00 1.00e+00 1.00e+00 1.00e+00 | BNμ(mean)=2.92e-02, BNσ(mean)=6.65e-01 | pred| μ(mean/min/max)=4.416e-03/-3.417e-02/2.984e-02 | σ(mean/min/max)=1.177e+00/1.168e+00/1.178e+00
[Iter 040] Loss: 1.7855 | lengthscale: N/A | outputscale: 6.100e-01 | noise: 7.818e-01 | feat| μ_abs(avg)=2.55e-05, σ(avg/min/max)=9.78e-01/9.72e-01/9.83e-01 | μ[:5]=4.44e-05 3.84e-05 1.56e-05 3.35e-05 -3.03e-05 | σ[:5]=9.77e-01 9.76e-01 9.76e-01 9.80e-01 9.79e-01 | BNμ(mean)=-1.38e-01, BNσ(mean)=9.71e-01 | pred| μ(mean/min/max)=4.154e-01/-1.598e+00/3.486e+00 | σ(mean/min/max)=1.010e+00/9.015e-01/1.180e+00
[Iter 080] Loss: 1.5247 | lengthscale: N/A | outputscale: 5.524e-01 | noise: 8.446e-01 | feat| μ_abs(avg)=2.70e-05, σ(avg/min/m

In [7]:
main_nngp_svgp_exact_ensemble_all()

Train X shape: (17397, 676)
Train y shape: (17397, 1)
[Ensemble] Training model 1/5
[Iter 001] Loss: 1.9569 | lengthscale: N/A | outputscale: 6.907e-01 | noise: 6.958e-01 | feat| μ_abs(avg)=1.30e-07, σ(avg/min/max)=1.00e+00/9.99e-01/1.00e+00 | μ[:5]=-5.12e-09 1.42e-07 -2.16e-07 -4.02e-08 -3.80e-08 | σ[:5]=1.00e+00 1.00e+00 1.00e+00 1.00e+00 1.00e+00 | BNμ(mean)=1.63e-03, BNσ(mean)=6.67e-01 | pred| μ(mean/min/max)=-1.560e-02/-6.476e-02/2.217e-02 | σ(mean/min/max)=1.188e+00/1.178e+00/1.189e+00
[Iter 040] Loss: 1.5192 | lengthscale: N/A | outputscale: 6.111e-01 | noise: 7.812e-01 | feat| μ_abs(avg)=4.36e-05, σ(avg/min/max)=9.73e-01/9.67e-01/9.78e-01 | μ[:5]=-1.10e-04 5.52e-05 3.44e-05 -1.81e-05 -6.23e-05 | σ[:5]=9.72e-01 9.76e-01 9.74e-01 9.73e-01 9.76e-01 | BNμ(mean)=5.65e-02, BNσ(mean)=1.01e+00 | pred| μ(mean/min/max)=-1.729e-01/-1.243e+00/9.834e-01 | σ(mean/min/max)=9.818e-01/9.059e-01/1.191e+00
[Iter 080] Loss: 1.4868 | lengthscale: N/A | outputscale: 5.684e-01 | noise: 8.257e-01 | fe

[{'name': 'uniform',
  'weights': array([0.2, 0.2, 0.2, 0.2, 0.2]),
  'valid_mse': 0.8706208505868375,
  'test_mse': 0.9129707594835123,
  'uq_metrics': {'alpha': 0.05,
   'empirical_coverage': 0.960919540229885,
   'avg_pred_std': 1.0099312676879668,
   'nll': 1.3820803206297934,
   'ce': 0.0011864220400386609,
   'spearman_err_unc': 0.0047587569113868074}},
 {'name': 'precision',
  'weights': array([0.18879133, 0.20225742, 0.20030842, 0.20713933, 0.20150351]),
  'valid_mse': 0.870974282358283,
  'test_mse': 0.9127073360923461,
  'uq_metrics': {'alpha': 0.05,
   'empirical_coverage': 0.9604597701149425,
   'avg_pred_std': 1.0095926800215023,
   'nll': 1.3819818959767383,
   'ce': 0.001177364874731418,
   'spearman_err_unc': 0.005108220468341497}},
 {'name': 'mse',
  'weights': array([0.2242949 , 0.15597722, 0.17955922, 0.17683878, 0.26332988]),
  'valid_mse': 0.8684663961237645,
  'test_mse': 0.9143814729391099,
  'uq_metrics': {'alpha': 0.05,
   'empirical_coverage': 0.96,
   'avg_pr